# Lesson3-01. Geocoding

## Overview

지오코딩은 주소를 좌표로 변환하거나 그 반대로 변환하는 것이고, 이는 정말 일반적인 GIS 작업이다. 파이썬에는 지오코딩을 정말 쉽게 해주는 라이브러리들이 있습니다. 바로 "geopy"인데, 이는 타사 지오코더와 다른 데이터 소스를 사용하여 전 세계의 주소, 도시, 국가 및 랜드마크의 좌표를 쉽게 찾을 수 있게 해준다.


## Geocoding in Geopandas

Geopandas에서 지오코딩을 수행할 수 있는데, 주소(문자열) 목록을 지오코딩하고 지오메트리 열에 결과인 점 객체를 포함하는 geodataframe을 반환할 수 있는 `geocode()`라는 함수가 있다.

In [6]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# .txt 파일 경로
fp = r"Dataset/Lesson3/address.txt"

# 데이터 읽기
data = pd.read_csv(fp, sep=";")
data.head()

,id,address
0,1000,"Itämerenkatu 14, 00101 Helsinki, Finland"
1,1001,"Kampinkuja 1, 00100 Helsinki, Finland"


## Table Join

테이블 조인은 GIS 분석을 할 때 정말 자주 해야 하는 것이다. Pandas/Geopandas에서는 .merge() - 함수를 사용하여 공통 키 속성을 기반으로 서로 다른 테이블의 데이터를 쉽게 결합할 수 있다.
이번에는 공통 열 주소를 기반으로 데이터 및 지오데이터 프레임을 함께 결합해 보겠다. 파라미터 "on"은 테이블의 공통 키를 결정하는 데 사용된다. 첫 번째 테이블의 키 이름이 다른 테이블과 다른 경우 left_on 및 right_on 매개 변수를 사용하여 각 테이블에 대해 키 이름을 개별적으로 지정할 수도 있다.
먼저 "geo"라는 Geometry를 가진 데이터를 만든 후 앞의 파일인 "data"와 조인하도록 하겠다.

In [23]:
geo = pd.DataFrame({"address": ["Itämerenkatu 14, 00101 Helsinki, Finland", "Kampinkuja 1, 00100 Helsinki, Finland"], "geometry": ["POINT (24.9146767 60.1628658)", "POINT (24.9301701 60.1683731)"]})
geo

,address,geometry
0,"Itämerenkatu 14, 00101 Helsinki, Finland",POINT (24.9146767 60.1628658)
1,"Kampinkuja 1, 00100 Helsinki, Finland",POINT (24.9301701 60.1683731)


In [24]:
join = geo.merge(data, on="address")
join.head()

,address,geometry,id
0,"Itämerenkatu 14, 00101 Helsinki, Finland",POINT (24.9146767 60.1628658),1000
1,"Kampinkuja 1, 00100 Helsinki, Finland",POINT (24.9301701 60.1683731),1001


그 결과 하나의 테이블로 열이 합쳐져 데이터화 된 것을 볼 수 있다.
이 데이터를 Shapefile로 저장할 수도 있다.

In [25]:
join = gpd.GeoDataFrame(join[["address", "id"]], crs="EPSG:4326", geometry=[Point(24.91476767, 60.1628658), Point(24.9301701, 60.1683731)])

# 결과 경로 설정하기
outfp = r"Dataset/Lesson3/addresses.shp"

# Shapefile로 저장하기
join.to_file(outfp)